In [62]:
import sys
from os import listdir
from os.path import isdir,isfile, join
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

from pybedtools import BedTool
import pybedtools

# DE genes

In [63]:
cm = pd.read_csv('/Users/minjunpark/Documents/hali/syntax_analysis/DE_genes/DEG_whole_heart_P1_vs_P8_MI_RNA_Cardiomyocytes.csv', index_col=0)
fb = pd.read_csv('/Users/minjunpark/Documents/hali/syntax_analysis/DE_genes/DEG_whole_heart_P1_vs_P8_MI_RNA_Fibroblasts.csv', index_col=0)
endo = pd.read_csv('/Users/minjunpark/Documents/hali/syntax_analysis/DE_genes/DEG_whole_heart_P1_vs_P8_MI_RNA_Endothelial_1.csv', index_col=0)

In [64]:
cm

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
Actb,0.000000e+00,-3.579699,0.639,0.953,0.000000e+00
Actc1,0.000000e+00,2.856948,0.998,0.338,0.000000e+00
Myl2,0.000000e+00,3.390666,0.995,0.336,0.000000e+00
Myl3,0.000000e+00,3.219670,0.993,0.315,0.000000e+00
Slc25a4,0.000000e+00,2.614645,0.998,0.690,0.000000e+00
...,...,...,...,...,...
Ybx1,1.422799e-11,-0.047966,0.900,0.797,3.553299e-07
Cdca8,3.153871e-11,-0.665434,0.177,0.266,7.876479e-07
mt-Rnr2,4.710665e-04,-0.212469,1.000,1.000,1.000000e+00
mt-Rnr1,1.509466e-03,-0.218356,0.960,0.949,1.000000e+00


In [65]:
## Filter df (adj p-val < 0.01 & abs(log2FC) > 1.25)
p_val_thres = 0.01
log2FC_thres = 1.25

cm = cm[cm.p_val_adj < p_val_thres]
cm_up = cm[cm.avg_log2FC > log2FC_thres]
cm_down = cm[cm.avg_log2FC < -log2FC_thres]

fb = fb[fb.p_val_adj < p_val_thres]
fb_up = fb[fb.avg_log2FC > log2FC_thres]
fb_down = fb[fb.avg_log2FC < -log2FC_thres]

endo = endo[endo.p_val_adj < p_val_thres]
endo_up = endo[endo.avg_log2FC > log2FC_thres]
endo_down = endo[endo.avg_log2FC < -log2FC_thres]

cm_up

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
Actc1,0.000000e+00,2.856948,0.998,0.338,0.000000e+00
Myl2,0.000000e+00,3.390666,0.995,0.336,0.000000e+00
Myl3,0.000000e+00,3.219670,0.993,0.315,0.000000e+00
Slc25a4,0.000000e+00,2.614645,0.998,0.690,0.000000e+00
Tnnt2,0.000000e+00,2.654837,0.995,0.437,0.000000e+00
...,...,...,...,...,...
Nppa,5.191542e-79,3.448894,0.605,0.111,1.296536e-74
Nmrk2,9.522836e-71,1.876591,0.395,0.013,2.378233e-66
Gm28438,9.283941e-68,1.302562,0.543,0.099,2.318572e-63
mt-Co1,8.753882e-65,2.053520,0.950,0.756,2.186195e-60


In [66]:
# Take an example TF bedfile (contains all the peaks --> now need to split into up & down regulated genes)
TF_bedfile = '/Users/minjunpark/Documents/hali/syntax_analysis/tiled_heatmap/ETS1_outs/A1JVI6.bed'
df = pd.read_csv(bedfile, header=None, sep='\t')
df

,0,1,2,3
0,chr17,23773962,23774645,0
1,chr1,154033735,154034801,0
2,chr11,75925691,75927350,0
3,chr6,124964605,124966195,0
4,chr6,39263595,39265341,0
...,...,...,...,...
124343,chr5,142894173,142910645,3
124344,chr1,73901265,73916772,0
124345,chr2,26569564,26600429,7
124346,chr2,76728332,76754932,4


In [ ]:
## What I need to do is find the interaction with the DE genes in CM, FB, or endotheliel...
## --> I don't really need to go over these steps?

In [67]:
genes_bed = '/Users/minjunpark/Documents/hali/syntax_analysis/influence_score/AllGenes_100kb_flanking_regions_dropped_duplicates.bed'
genes = pd.read_csv(genes_bed, sep='\t', header=None)
#subs = genes[genes.iloc[:,0].isin(cm_up.index.tolist())].iloc[:,0].reset_index(0, drop=True)
cm_up_bed = genes[genes.iloc[:,0].isin(cm_up.index.tolist())].reset_index(0, drop=True)
cm_up_bed = cm_up_bed.drop(columns=[0])
cm_up_bed.to_csv('cm_up.bed', sep='\t', index=False, header=None)

cm_down_bed = genes[genes.iloc[:,0].isin(cm_down.index.tolist())].reset_index(0, drop=True)
cm_down_bed = cm_down_bed.drop(columns=[0])
cm_down_bed.to_csv('cm_down.bed', sep='\t', index=False, header=None)

In [68]:
TF_bed = BedTool(TF_bedfile)
cm_up_bed = BedTool('cm_up.bed')
cm_down_bed = BedTool('cm_down.bed')

In [69]:
intersected_up = TF_bed.intersect(cm_up_bed)
df_cm_up = pd.read_table(intersected_up.fn, names=['chrom', 'start', 'stop', 'name', 'score', 'strand'])
df_cm_up = df_cm_up.drop(columns=['name', 'score', 'strand'])

intersected_down = TF_bed.intersect(cm_down_bed)
df_cm_down = pd.read_table(intersected_down.fn, names=['chrom', 'start', 'stop', 'name', 'score', 'strand'])
df_cm_down = df_cm_down.drop(columns=['name', 'score', 'strand'])

df_cm_up

,chrom,start,stop
0,chr17,23773962,23774645
1,chr11,70017290,70022230
2,chr14,34538739,34593970
3,chr7,140950604,140956978
4,chr1,36554114,36561687
...,...,...,...
8808,chr8,13239782,13245787
8809,chr12,112670292,112676243
8810,chr2,32708398,32715639
8811,chr5,122099921,122115703


In [70]:
df_cm_down

,chrom,start,stop
0,chr7,140950604,140956978
1,chr7,140950604,140956978
2,chr6,135362159,135366189
3,chr7,142476538,142489232
4,chr6,30652586,30653307
...,...,...,...
9296,chr2,32708398,32715639
9297,chr5,142894173,142910645
9298,chr5,142894173,142910645
9299,chr2,26569564,26600429


# Loop over DE genes

In [75]:
cm_up_genes = genes[genes.iloc[:,0].isin(cm_up.index.tolist())].reset_index(0, drop=True)
cm_down_genes = genes[genes.iloc[:,0].isin(cm_down.index.tolist())].reset_index(0, drop=True)
cm_up_genes

,0,1,2,3
0,Ankrd23,chr1,36435739,36635739
1,Fhl2,chr1,43063961,43263961
2,Hspd1,chr1,54988024,55188024
3,Ndufs1,chr1,63044961,63244961
4,Acadl,chr1,66758331,66958331
...,...,...,...,...
231,Idh3g,chrX,73680696,73880696
232,Cox7b,chrX,105915700,106115700
233,Rnf128,chrX,139463316,139663316
234,Smpx,chrX,157598910,157798910


In [77]:
# CM1 down-regulated in P1
! mkdir -p cm_down
for i in range(len(cm_up_genes)):
    num = 0
    gene = cm_up_genes.iloc[i,0]
    chromosome = cm_up_genes.iloc[i,1]
    start = cm_up_genes.iloc[i,2]
    end = cm_up_genes.iloc[i,3]
    r1 = set(range(start, end))
    with open('./cm_down/{}.bed'.format(gene), 'w') as f:
    
        for j in range(len(df_cm_down)):
            chromosome2 = df_cm_down.iloc[j,0]
            start2 = df_cm_down.iloc[j,1]
            end2 = df_cm_down.iloc[j,2] 
            r2 = set(range(start2, end2))

            if chromosome == chromosome2 and len(r1.intersection(r2)) > 0:
                f.write('{}\t{}\t{}\n'.format(chromosome2, start2, end2))
                num += 1
                
    if num > 0:
        df = pd.read_csv('./cm_down/{}.bed'.format(gene), sep='\t', header=None)
        df = df.drop_duplicates() # remove duplicate rows
        df.to_csv('./cm_down/{}.bed'.format(gene), sep='\t', header=None, index=None)


# LA down CM1

In [6]:
#sns.set_theme(color_codes=True)
df_down = pd.read_csv('/Users/minjunpark/Documents/Pitx2/manuscript/fig3/data/LAdown_bicluster_outs/A1JVI6.bed', header=None, sep='\t')
df_down

,0,1,2,3
0,chr4,43005833,43006305,0
1,chr13,12257146,12258509,0
2,chr16,4474490,4475757,0
3,chr3,142345065,142346447,0
4,chr4,86083756,86084229,0
...,...,...,...,...
118,chr10,95551851,95553216,0
119,chr13,12326713,12328185,0
120,chr16,4511271,4513209,0
121,chr1,55074064,55075901,0


In [5]:
#LAdown_gene_list = ['Myoz2', 'Jph2', 'Actn2', 'Stat3', 'Atp1a1', 'Pdlim5', 'Asb2']
LA_gene_list = list(LAcm.index)
LA_genes = pd.read_csv('/Users/minjunpark/Documents/Pitx2/manuscript/fig3/jeff/LA_genes_100Kb_expressed_adjp(0.01).bed', sep='\t', header=None)
#LAdown_genes = LAdown_genes[LAdown_genes[0].isin(LAdown_gene_list)]
LA_genes

,0,1,2,3
0,Ckm,chr7,19311094,19511095
1,Appl2,chr10,83548663,83748664
2,Tmem108,chr9,103661836,103861837
3,Tfdp2,chr9,96096275,96296276
4,Sgk3,chr1,9698130,9898131
...,...,...,...,...
214,Aldoa,chr7,126699177,126899178
215,Atp5g1,chr11,95975630,96175631
216,Ndufb9,chr15,58833810,59033811
217,Uqcr11,chr10,80306820,80506821


In [6]:
! mkdir -p LA_all
for i in range(len(LA_genes)):
    num = 0
    gene = LA_genes.iloc[i,0]
    chromosome = LA_genes.iloc[i,1]
    start = LA_genes.iloc[i,2]
    end = LA_genes.iloc[i,3]
    r1 = set(range(start, end))
    with open('./LA_all/{}.bed'.format(gene), 'w') as f:
    
        for j in range(len(df_down)):
            chromosome2 = df_down.iloc[j,0]
            start2 = df_down.iloc[j,1]
            end2 = df_down.iloc[j,2] 
            r2 = set(range(start2, end2))

            if chromosome == chromosome2 and len(r1.intersection(r2)) > 0:
                f.write('{}\t{}\t{}\n'.format(chromosome2, start2, end2))
                num += 1
                
    if num > 0:
        df = pd.read_csv('./LA_all/{}.bed'.format(gene), sep='\t', header=None)
        df = df.drop_duplicates() # remove duplicate rows
        df.to_csv('./LA_all/{}.bed'.format(gene), sep='\t', header=None, index=None)
